In [1]:
using Compat
using QPControl
using RigidBodyDynamics
using RigidBodyDynamics.PDControl
using RigidBodyDynamics.Contact
using StaticArrays
using AtlasRobot
using BenchmarkTools
using Compat.Test
using RigidBodySim

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9ed50fe60e4c3cfba5d8d51afc0a1ba695b21d72-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9ed50fe60e4c3cfba5d8d51afc0a1ba695b21d72-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

INFO: Loading HttpServer methods...


In [2]:
# load URDF
mechanism = AtlasRobot.mechanism()
remove_fixed_tree_joints!(mechanism);

In [3]:
# add environment
rootframe = root_frame(mechanism)
ground = HalfSpace3D(Point3D(rootframe, 0., 0., 0.), FreeVector3D(rootframe, 0., 0., 1.))
add_environment_primitive!(mechanism, ground);

In [4]:
# create optimizer
using MathOptInterface
using OSQP.MathOptInterfaceOSQP
const MOI = MathOptInterface
optimizer = OSQPOptimizer()
MOI.set!(optimizer, OSQPSettings.Verbose(), false)
MOI.set!(optimizer, OSQPSettings.EpsAbs(), 1e-5)
MOI.set!(optimizer, OSQPSettings.EpsRel(), 1e-5)
MOI.set!(optimizer, OSQPSettings.MaxIter(), 5000)
MOI.set!(optimizer, OSQPSettings.AdaptiveRhoInterval(), 25) # required for deterministic behavior

In [5]:
# create low level controller
const num_basis_vectors = 4
const lowlevel = MomentumBasedController{num_basis_vectors}(mechanism, optimizer);
for body in bodies(mechanism)
    for point in RigidBodyDynamics.contact_points(body)
        position = location(point)
        normal = FreeVector3D(default_frame(body), 0.0, 0.0, 1.0)
        μ = point.model.friction.μ
        contact = addcontact!(lowlevel, body, position, normal, μ)
        contact.maxnormalforce[] = 1e6 # TODO
        contact.weight[] = 1e-3
    end
end

In [6]:
# state initialization
function initialize!(state::MechanismState)
    mechanism = state.mechanism
    zero!(state)
    kneebend = 1.1
    hipbendextra = 0.1
    for sideprefix in ('l', 'r')
        knee = findjoint(mechanism, "$(sideprefix)_leg_kny")
        hippitch = findjoint(mechanism, "$(sideprefix)_leg_hpy")
        anklepitch = findjoint(mechanism, "$(sideprefix)_leg_aky")
        set_configuration!(state, knee, [kneebend])
        set_configuration!(state, hippitch, [-kneebend / 2 + hipbendextra])
        set_configuration!(state, anklepitch, [-kneebend / 2 - hipbendextra])
    end
    floatingjoint = first(out_joints(root_body(mechanism), mechanism))
    set_configuration!(state, floatingjoint, [1; 0; 0; 0; 0; 0; 0.85])
    state
end

initialize! (generic function with 1 method)

In [7]:
# create standing controller
feet = findbody.(mechanism, ["l_foot", "r_foot"])
pelvis = findbody(mechanism, "pelvis")
nominalstate = MechanismState(mechanism)
initialize!(nominalstate)
controller = StandingController(lowlevel, feet, pelvis, nominalstate);

In [8]:
state = MechanismState(mechanism)
initialize!(state)
τ = similar(velocity(state));
benchresult = @benchmark controller($τ, 0.0, $state)
@test benchresult.allocs == 0
benchresult

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     924.835 μs (0.00% GC)
  median time:      959.212 μs (0.00% GC)
  mean time:        988.118 μs (0.00% GC)
  maximum time:     1.388 ms (0.00% GC)
  --------------
  samples:          5042
  evals/sample:     1

In [9]:
# function test(τ, state, n)
#     for i = 1 : n
#         controller(τ, 0.0, state)
#     end
# end
# test(τ, state, 1)
# Profile.clear()
# @profile test(τ, state, 10000)
# using ProfileView
# ProfileView.view()

In [10]:
# set up visualizer
using MeshCat
using MeshCatMechanisms
if !isdefined(:gui)
    visuals = URDFVisuals(AtlasRobot.urdfpath(); package_path = [AtlasRobot.packagepath()])
    gui = GUI(mechanism, visuals)
    set_configuration!(gui.visualizer, configuration(nominalstate))
    open(gui)
end

In [11]:
# simulate
state = MechanismState(mechanism)
initialize!(state)
Δt = 1 / 500
pcontroller = PeriodicController(similar(velocity(state)), Δt, controller)
# TODO: add damping
dynamics = Dynamics(mechanism, pcontroller)
problem = ODEProblem(dynamics, state, (0., 10.))
sol = solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6, callback = CallbackSet(gui));

In [12]:
@time sol = solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6)
@test sol.retcode == :Success
copyto!(state, last(sol.u))
@test norm(velocity(state)) ≈ 0 atol=1e-8
@test center_of_mass(state).v[3] > 1

Test Passed

 10.435575 seconds (814.83 k allocations: 74.915 MiB, 3.15% gc time)


In [13]:
setanimation!(gui.visualizer, sol);